In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn.functional as F

import sys
sys.path.append('../dgmr-freq-mod')
sys.path.append('../')
import dgmr

import common.utils as utils

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [5]:
ccs = dgmr.common.ContextConditioningStack(
    input_channels=64,
    conv_type='standard',
    output_channels=64*4,
)

lcs = dgmr.common.LatentConditioningStack(
    shape=(8 * 1, 128 // 32, 128 // 32),
    output_channels=384,
)

sampler = dgmr.generators.Sampler(
    forecast_steps=5,
    latent_channels=384,
    context_channels=64*4,
    output_channels=64*4,
)

model = dgmr.generators.Generator(ccs, lcs, sampler)

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model)} trainable parameters')

The model has 12147209 trainable parameters


In [7]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
x = torch.randn(3, 4, 1, 128, 128)

In [9]:
o = model(x)

/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [10]:
y = torch.randn(3, FORECAST, 128, 128)
loss = F.mse_loss(y,o)

In [11]:
loss

tensor(1.1409, grad_fn=<MseLossBackward0>)

In [12]:
loss.backward()

In [13]:
opt.step()